### Import

In [12]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [13]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

In [14]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [15]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [16]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [17]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [18]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [19]:
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd

# 1️⃣ 결측 여부 Feature 추가
for col in numeric_columns:
    X_train_encoded[col + '_missing'] = X_train_encoded[col].isna().astype(int)
    X_test_encoded[col + '_missing'] = X_test_encoded[col].isna().astype(int)

# zero_imputer = SimpleImputer(strategy="constant", fill_value=0)
# X_train_encoded[numeric_columns] = zero_imputer.fit_transform(X_train_encoded[numeric_columns])
# X_test_encoded[numeric_columns] = zero_imputer.transform(X_test_encoded[numeric_columns])

from numpy import log1p
# 🔹 로그 변환 적용 (Skewed Data Handling)
skewed_cols = ['총 생성 배아 수', '수집된 신선 난자 수', '저장된 배아 수', '미세주입된 난자 수']

for col in skewed_cols:
    X_train_encoded[col + '_log'] = log1p(X_train_encoded[col])
    X_test_encoded[col + '_log'] = log1p(X_test_encoded[col])

# 3️⃣ 80% 이상 결측치가 있는 컬럼 제거
missing_ratio = X_train_encoded.isnull().mean()
high_missing_columns = missing_ratio[missing_ratio > 0.8].index.tolist()
X_train_encoded.drop(columns=high_missing_columns, inplace=True)
X_test_encoded.drop(columns=high_missing_columns, inplace=True)

# 4️⃣ Feature Engineering (특성 추가)
X_train_encoded['배아_생성_효율'] = X_train_encoded['총 생성 배아 수'] / (X_train_encoded['수집된 신선 난자 수'] + 1)
X_test_encoded['배아_생성_효율'] = X_test_encoded['총 생성 배아 수'] / (X_test_encoded['수집된 신선 난자 수'] + 1)

X_train_encoded['배아_저장_비율'] = X_train_encoded['저장된 배아 수'] / (X_train_encoded['총 생성 배아 수'] + 1)
X_test_encoded['배아_저장_비율'] = X_test_encoded['저장된 배아 수'] / (X_test_encoded['총 생성 배아 수'] + 1)

X_train_encoded['난자_배아_비율'] = X_train_encoded['미세주입된 난자 수'] / (X_train_encoded['총 생성 배아 수'] + 1)
X_test_encoded['난자_배아_비율'] = X_test_encoded['미세주입된 난자 수'] / (X_test_encoded['총 생성 배아 수'] + 1)


# 5️⃣ 이상치 처리 (Clip 적용)
for col in ['총 생성 배아 수', '수집된 신선 난자 수', '저장된 배아 수']:
    X_train_encoded[col] = X_train_encoded[col].clip(lower=1, upper=50)
    X_test_encoded[col] = X_test_encoded[col].clip(lower=1, upper=50)

# 6️⃣ Feature Scaling 제거 (트리 모델은 불필요)

import seaborn as sns
import matplotlib.pyplot as plt

# 상관 행렬 계산
corr_matrix = X_train_encoded.corr()

# 높은 상관관계(절대값 0.9 이상)를 가지는 변수 찾기
high_corr_features = set()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.9:
            colname = corr_matrix.columns[i]
            high_corr_features.add(colname)

print("높은 상관관계를 가지는 Feature:", high_corr_features)

# 제거
X_train_encoded.drop(columns=high_corr_features, inplace=True)
X_test_encoded.drop(columns=high_corr_features, inplace=True)


높은 상관관계를 가지는 Feature: {'해동 난자 수_missing', 'IVF 시술 횟수', '착상 전 유전 진단 사용 여부', '미세주입 배아 이식 수_missing', '미세주입된 난자 수_missing', 'IVF 임신 횟수', '저장된 배아 수_missing', 'IVF 출산 횟수', '대리모 여부', '배아 해동 경과일_missing', '기증자 정자와 혼합된 난자 수_missing', '미세주입에서 생성된 배아 수_missing', '배란 유도 유형', '이식된 배아 수_missing', '저장된 신선 난자 수_missing', '난자 채취 경과일_missing', '기증 배아 사용 여부', '미세주입에서 생성된 배아 수', '부부 주 불임 원인', '해동된 배아 수_missing', '총 생성 배아 수_missing', '미세주입 후 저장된 배아 수_missing', '미세주입된 난자 수_log', '파트너 정자와 혼합된 난자 수_missing', '수집된 신선 난자 수_missing', '파트너 정자와 혼합된 난자 수', '혼합된 난자 수_missing'}


### 검증

In [20]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
import optuna
from sklearn.metrics import roc_auc_score 
from tqdm import tqdm

# Optuna 진행 상황을 추적하는 tqdm 콜백 함수
class TQDMCallback:
    def __init__(self, total):
        self.pbar = tqdm(total=total, desc="Optuna Hyperparameter Tuning", position=0, leave=True)

    def __call__(self, study, trial):
        self.pbar.update(1)

    def close(self):
        self.pbar.close()

# 하이퍼파라미터 튜닝 (Optuna)
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0),
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'random_state': 42
    }
    
    model = xgb.XGBClassifier(**params)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = []
    
    for train_idx, val_idx in cv.split(X_train_encoded, y):
        X_train_fold, X_val_fold = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(
            X_train_fold,
            y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            verbose=False
        )
        y_val_pred = model.predict_proba(X_val_fold)[:, 1]
        
        auc_score = roc_auc_score(y_val_fold, y_val_pred)
        cv_scores.append(auc_score)
    
    return np.mean(cv_scores)

# Optuna 최적화 실행 (시도 횟수: 50번, tqdm 추가)
n_trials = 50
tqdm_callback = TQDMCallback(total=n_trials)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=n_trials, callbacks=[tqdm_callback])
tqdm_callback.close()

# 최적 하이퍼파라미터 출력
best_params = study.best_params
print(f"✅ Optuna 최적화 완료! 최적 하이퍼파라미터: {best_params}")

# 최적 하이퍼파라미터 적용
# (XGBoost는 random_state를 명시적으로 다시 지정하는 것이 좋습니다.)
model = xgb.XGBClassifier(**best_params, random_state=42)

# 교차 검증 설정 (5-Fold)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 진행도 표시를 위한 tqdm 적용
cv_scores = []
for fold, (train_idx, val_idx) in enumerate(tqdm(cv.split(X_train_encoded, y), desc="Cross Validation Progress", total=5)):
    X_train_fold, X_val_fold = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
    
    model.fit(
        X_train_fold,
        y_train_fold,
        eval_set=[(X_val_fold, y_val_fold)],
        verbose=False
    )
    y_val_pred = model.predict_proba(X_val_fold)[:, 1]
    
    auc_score = roc_auc_score(y_val_fold, y_val_pred)
    cv_scores.append(auc_score)
    
    print(f"Fold {fold+1}: ROC-AUC = {auc_score:.4f}")

# 최종 결과 출력
print(f"\n✅ 5-Fold ROC-AUC 점수 평균: {np.mean(cv_scores):.4f}")
print(f"각 Fold 점수: {cv_scores}")

Optuna Hyperparameter Tuning:   0%|          | 0/50 [00:00<?, ?it/s][I 2025-02-03 02:50:35,854] A new study created in memory with name: no-name-51aafff9-5ef4-4328-9699-59e0bb9016dc
[I 2025-02-03 02:51:16,783] Trial 0 finished with value: 0.7373527621262215 and parameters: {'n_estimators': 662, 'max_depth': 9, 'learning_rate': 0.018423208602250003, 'reg_lambda': 3.6293412499217794}. Best is trial 0 with value: 0.7373527621262215.
Optuna Hyperparameter Tuning:   0%|          | 0/50 [02:49<?, ?it/s].79s/it]
[I 2025-02-03 02:52:36,405] Trial 2 finished with value: 0.6956604172554173 and parameters: {'n_estimators': 825, 'max_depth': 12, 'learning_rate': 0.24263527713372918, 'reg_lambda': 7.695407587194057}. Best is trial 0 with value: 0.7373527621262215.
Optuna Hyperparameter Tuning: 100%|██████████| 50/50 [26:34<00:00, 31.89s/it]


✅ Optuna 최적화 완료! 최적 하이퍼파라미터: {'n_estimators': 585, 'max_depth': 4, 'learning_rate': 0.040314325768186714, 'reg_lambda': 4.9869088471461165}


Fold 1: ROC-AUC = 0.7378


Fold 2: ROC-AUC = 0.7427


Fold 3: ROC-AUC = 0.7399


Fold 4: ROC-AUC = 0.7376


Cross Validation Progress: 100%|██████████| 5/5 [00:20<00:00,  4.19s/it]

Fold 5: ROC-AUC = 0.7411

✅ 5-Fold ROC-AUC 점수 평균: 0.7398
각 Fold 점수: [0.737842502513753, 0.7427190621099498, 0.7398947613739844, 0.737643518351006, 0.7410521826786194]


### Train

In [27]:
# model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y);

### Predict

In [28]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [29]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba

In [30]:
sample_submission.to_csv('./baseline_submit.csv', index=False)